# Load Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install dependent libraries

In [ ]:
!pip install flask
!pip install pyngrok

# Upload weights of trained models for each language

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


asl_model = load_model('/path/to/asl/model.keras')
isl_model = load_model('/path/to/isl/model.keras')

# Build Flask App

In [ ]:
import os
from flask import Flask, render_template, request, send_from_directory

In [ ]:
# function to preprocess image before feeding into CNN
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(100, 100))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# predict the class to which uploaded image belongs to
def predict_class(image_path, class_labels, model):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    probability = prediction[0][predicted_class]
    return list(class_labels.keys())[list(class_labels.values()).index(predicted_class)], probability, prediction[0]

def prediction(filename, model):
    classes = [str(i) for i in range(10)] + [chr(ord('a') + i) for i in range(26)]

    class_labels = {value:index for index, value in enumerate(classes)}
    predicted_class, probability, class_probabilities = predict_class(image_path=filename, class_labels=class_labels, model=model)

    return predicted_class, probability

In [ ]:
# initiating Flask App
app = Flask(__name__, template_folder='/templates', static_folder='/static')
app.config['UPLOAD_FOLDER'] = '/static/uploads/'


In [ ]:
import re

# function to create folder to store uploaded images
def create_upload_folder():
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])

# preprocess string before display
def removeSpecialCharacters(string):
    return re.sub(r'[^a-zA-Z0-9]', '', string)

def formatString(string):
    string = string.lower()
    strings = [removeSpecialCharacters(s) for s in string.split()]
    return strings

In [ ]:
# creating routes for Flask App

# route to save uploaded file to folder
@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# route for the landing page
@app.route('/')
def index():
  return render_template('index.html')

# route for image recognition and translation
@app.route('/recognition')
def recognition():
    return render_template('recognition.html')

# route for the output of image recognition
@app.route('/upload', methods=['POST'])
def upload_image():
    create_upload_folder()
    language = request.form['options']
    if 'file' not in request.files:
        return "No file part"
    file = request.files['file']
    if file.filename == '':
        return "No selected file"
    if file:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filepath)
        if language == "asl":
          class_name, probability = prediction(filepath, model=asl_model)
        else:
          class_name, probability = prediction(filepath, model=isl_model)
        return render_template('result.html', filename=file.filename, class_name=class_name, probability = round(probability*100, 2), language=language)

# route for translation from text to sign language
@app.route('/fingerspell')
def fingerSpell():
    return render_template('fingerSpell.html')

# route for the output of translation
@app.route('/fingerresult', methods=['POST'])
def fingerResult():
    input_string = request.form['inputString'].lower()
    language = request.form['selectLanguage'].lower()
    strings = formatString(input_string)

    return render_template('fingerResult.html', inputString=strings, language=language)


# Setting up ngrok to expose app

In [ ]:
from pyngrok import ngrok

# set your Ngrok authtoken
ngrok.set_auth_token("auth-token")

# open a HTTP tunnel on the default port 5000
public_url = ngrok.connect(5000, "http")

print("Ngrok Tunnel:", public_url)


In [ ]:
if __name__ == "__main__":
  app.run()